
# Least-Squares Monte Carlo for American Options - Python

---
Ouijdane ABCHIR, January 2024.
---
About this notebook...
---

- **Summary** : This notebook presents a Python implementation of the well-known **Least-Square Monte Carlo (LSMC) method** for the valuation of **American Options** as it appears  in the **Longstaff-Schwartz (2001)** paper.

- **Reference**: [Longstaff-Schwartz (2001): "Valuing American Options by Simulation: A Simple Least-Squares Approach." Review of Financial Studies, Vol. 14, 113-147](https://www.google.de/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=0CCEQFjAAahUKEwiXtNSZm4rHAhXHOhQKHTjBD3k&url=https%3A%2F%2Fpeople.math.ethz.ch%2F~hjfurrer%2Fteaching%2FLongstaffSchwartzAmericanOptionsLeastSquareMonteCarlo.pdf&ei=7PO9VZeOBcf1ULiCv8gH&usg=AFQjCNFQr1r_Cf_pxylg_amU3TFOZVDc8w&sig2=ixZnX_wWQ48G66BMuQTPZA&bvm=bv.99261572,d.d24)

## Imports

In [13]:
import IPython
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sys import version

----------Least-Squares MC for American Options : Conditions for Replication---------
Python version :  3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]
Numpy version :  1.24.3
IPython version :  8.17.2
-------------------------------------------------------------------------------------


---
# A Python Class for American Options pricing using LSMC
---

The basic idea of the **Longstaff-Schwartz** algorithm, described in detail in **Longstaff and Schwartz (2001)** paper, is to use least-squares regression on a finite set of functions as a proxy for conditional expectation estimates.

First of all, the time axis has to be discretized -i.e., if the American option is alive within the time horizon $[0,T]$, early exercise is only allowed at discrete times $0 < t_1 < t_2 <...< t_J=T$. The American option is thus approximated by a Bermudan option. For a particular exercise date $t_k$, early exercise is performed if the payoff from immediate exercise exceeds the continuation value -i.e., the value of the (remaining) option if it is not exercised at $t_k$. This continuation value can be expressed as conditional expectation of the option payoff with respect to the risk-neutral pricing measure $Q$.

- The initial step of the actual algorithm is to determine the cashflow vector $C_{t_J}$ at the last timestep $t_J$. These cashflows are easy to get because the continuation values are then zero, or in other words, it is directly the payoff of a vanilla call option in the terminal value of each simulation $i$:
$$ C_{i,t_J} = (S_{i, t_J} - K)_+ $$
- Second, we consider the spot prices at time step $t_{J-1}$, and estimate the **exercise value**, selecting those for which has positive payoff, or :
$$(S_{i, t_{J-1}} - K)_+ > 0$$
- In order to obtain the mentioned **continuation values**, **Longstaff-Schwartz** regress the discounted future cashflows realized from continuing onto a finite set of basis functions of our values for the spot price. The regression is done by using the valeus from all of the paths. The set of the basis functions for the regression, in this notebook, is a polynomial regression (of 5 degrees) but it could also be *Hermite, Legendre, Chebyshev, Gegenbauer, or Jacobi polynomials*, for example. If $S$ is the spot price, $a_j$ are coefficients and $B_j$ is the set of basis functions, then the **continuation value** for a path $i$ with values $S_{i, t_{n}}$ at time $t_n$ is :
$$Cont_{i,t_{n-1}} = \sum_{j=0}^{\infty} a_j(t_n) B_j(S_{i, t_{n}})$$
- Once we have the **continuation values** and **exercise values**, we will perform **early exercise condition of the american option** whenever :
$$C_{i,t_{n-1}} > Cont_{i,t_{n-1}}$$
- And finally, we then **step backward** through time, until we reach the *first time step*. At each time step, early exercise is performed as described previously. Note that whenever a cashflow at timestep $t_k$ is generated by early exercise in path $i$, all cashflows that occur in this path lated than $t_k$ (this is, at most, one) have to be removed.
- Once the whole backward process reach the initial point, we can build a cashflow or **value matrix** from the cashflow vectors $C_{i,t_{n}}$ by concatenating the cashflow vectors $C_{i,t_{n}}$, $n = 1,\ldots,J$, and the option value is given by the arithmetic average of the row sums.

In [72]:
class AmericanOptionsLSMC(object):
    """ Class for American options pricing using Longstaff-Schwartz (LSMC)
    S0 : float : initial stock/index level
    strike : float : strike price
    T : float : time to maturity (in year fractions)
    M : int : grid or granularity for time (in number of total points)
    r : float : constant risk-free short rate
    div : float : dividend yield
    sigma : float : volatility fator in diffusion term
    simulations : int : number of simulated paths
    """
    
    def __init__(self, option_type, S0, strike, T, M, r, div, sigma, simulations):
        
        try :
            self.option_type = option_type
            assert isinstance(option_type, str)
            assert S0 >= 0
            self.S0 = float(S0)
            self.strike = float(strike)
            assert T > 0
            self.T = float(T)
            assert M > 0
            self.M = int(M)
            assert r >= 0
            self.r = float(r)
            assert div >= 0
            self.div = float(div)
            assert sigma > 0
            self.sigma = float(sigma)
            assert simulations > 0
            self.simulations = int(simulations)
        
        except ValueError :
            print('Error passing Options parameters')
            
        if option_type != 'call' and option_type != 'put' :
            raise ValueError("Error: option type not valid. Enter 'call' or 'put'")
        
        if S0 < 0 or strike < 0 or T <= 0 or r < 0 or div < 0 or sigma < 0:
            raise ValueError("Error : Negative inputs not allowed")
            
        self.time_unit = self.T / float(self.M)
        self.discount = np.exp(-self.r * self.time_unit)
        
    @property
    def MCprice_matrix(self, seed = 123):
        " Returns MC price matrix rows : time columns : price-path simulation"
        np.random.seed(seed)
        MCprice_matrix = np.zeros((self.M+1, self.simulations), dtype = np.float64)
        MCprice_matrix[0,:] = self.S0
        for t in  range(1, self.M + 1):
            brownian = np.random.standard_normal(self.simulations // 2)
            brownian = np.concatenate((brownian, -brownian))
            MCprice_matrix[t,:] = (MCprice_matrix[t-1, :]
                                  * np.exp((self.r - self.sigma **2 / 2.0) * self.time_unit
                                  + self.sigma * brownian * np.sqrt(self.time_unit)))
        return MCprice_matrix
    
    @property
    def MCpayoff(self):
        "Returns the inner-value of American Option"
        if self.option_type == 'call':
            payoff = np.maximum(self.MCprice_matrix - self.strike, 
                               np.zeros((self.M+1,self.simulations), dtype = np.float64))
        else :
            payoff = np.maximum(self.strike - self.MCprice_matrix, 
                               np.zeros((self.M+1,self.simulations), dtype = np.float64))
        
        return payoff
    
    @property
    def value_vector(self):
        value_matrix = np.zeros_like(self.MCpayoff)
        value_matrix[-1,:] = self.MCpayoff[-1,:]
        for t in range(self.M-1, 0, -1):
            regression = np.polyfit(self.MCprice_matrix[t,:], value_matrix[t+1,:] * self.discount, 5) 
            continuation_value = np.polyval(regression, self.MCprice_matrix[t,:])
            value_matrix[t,:] = np.where(self.MCpayoff[t,:] > continuation_value, 
                                         self.MCpayoff[t,:],
                                         value_matrix[t+1,:] * self.discount)
        
        return value_matrix[1,:] * self.discount
    
    @property
    def price(self):
        return np.sum(self.value_vector) / float(self.simulations)
    
    @property
    def delta(self):
        diff = self.S0 * 0.01
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0 + diff, self.strike, self.T, self.M, self.r, self.div, self.sigma, self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0 - diff, self.strike, self.T, self.M, self.r, self.div, self.sigma, self.simulations)
        
        return (myCall_1.price - myCall_2.price) / float(2.0 * diff)
    
    @property
    def gamma(self):
        diff = self.S0 * 0.01
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0 + diff, self.strike, self.T, self.M, self.r, self.div, self.sigma, self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0 - diff, self.strike, self.T, self.M, self.r, self.div, self.sigma, self.simulations)
                
        return  (myCall_1.delta - myCall_2.delta) / float(2.0 * diff)
    
    @property
    def vega(self):
        diff = self.sigma * 0.01
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T, self.M, self.r, self.div, self.sigma + diff, self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T, self.M, self.r, self.div, self.sigma - diff, self.simulations)
        
        return (myCall_1.price - myCall_2.price) / float(2.0 * diff)
    
    @property
    def rho(self):
        diff = self.r * 0.01
        if (self.r - diff) <0:
            myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T, self.M, self.r + diff, self.div, self.sigma, self.simulations)
            myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T, self.M, self.r, self.div, self.sigma, self.simulations)
        
            return (myCall_1.price - myCall_2.price) / float(diff)
        else :
            myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T, self.M, self.r + diff, self.div, self.sigma, self.simulations)
            myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T, self.M, self.r - diff, self.div, self.sigma, self.simulations)
        
            return (myCall_1.price - myCall_2.price) / float(2.0 * diff)
    
    @property
    def theta(self):
        diff = 1/252
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T + diff, self.M, self.r, self.div, self.sigma, self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T - diff, self.M, self.r, self.div, self.sigma, self.simulations)
        
        return (myCall_2.price - myCall_1.price) / float(2.0 * diff)

In [68]:
AmericanPUT = AmericanOptionsLSMC('put', 36.0, 40.0, 1.0, 50, 0.06, 0.06, 0.2, 10000)

In [71]:
AmericanPUT = AmericanOptionsLSMC('put', 36.0, 40.0, 1.0, 50, 0.06, 0.06, 0.2, 10000)
print('Price: ', AmericanPUT.price)
print('Delta: ', AmericanPUT.delta)
print('Gamma: ', AmericanPUT.gamma)
print('Vega: ', AmericanPUT.vega)
print('Rho: ', AmericanPUT.rho)
print('Theta: ', AmericanPUT.theta)

Price:  4.473117701771221
Delta:  -0.7112251324731934
Gamma:  0.12615233203125087
Vega:  12.196835824506369
Rho:  -10.0335229852333
Theta:  -1.8271728267244622


In [76]:
def prices():
    for S0 in (36.0, 38.0, 40.0, 42.0, 44.0) :     # initial stock price values
        for vol in (0.2, 0.4):                     # volatility values
            for T in (1.0, 2.0) :                  # times to maturity
                AmericanPUT = AmericanOptionsLSMC('put',S0, 40., T, 50, 0.06, 0.06, vol, 1500)
                print('Initial price : %4.1f, Sigma : %4.2f, Expire : %4.1f --> Option Value %4.3f'% (S0, vol, T, AmericanPUT.price))

In [77]:
from time import time
t0 = time()
optionValues = prices()
t1 = time()
d1 = t1 - t0
print('Duration in Seconds %6.3f' % d1)

Initial price : 36.0, Sigma : 0.20, Expire :  1.0 --> Option Value 4.439
Initial price : 36.0, Sigma : 0.20, Expire :  2.0 --> Option Value 4.779
Initial price : 36.0, Sigma : 0.40, Expire :  1.0 --> Option Value 7.135
Initial price : 36.0, Sigma : 0.40, Expire :  2.0 --> Option Value 8.459
Initial price : 38.0, Sigma : 0.20, Expire :  1.0 --> Option Value 3.225
Initial price : 38.0, Sigma : 0.20, Expire :  2.0 --> Option Value 3.726
Initial price : 38.0, Sigma : 0.40, Expire :  1.0 --> Option Value 6.134
Initial price : 38.0, Sigma : 0.40, Expire :  2.0 --> Option Value 7.666
Initial price : 40.0, Sigma : 0.20, Expire :  1.0 --> Option Value 2.296
Initial price : 40.0, Sigma : 0.20, Expire :  2.0 --> Option Value 2.808
Initial price : 40.0, Sigma : 0.40, Expire :  1.0 --> Option Value 5.201
Initial price : 40.0, Sigma : 0.40, Expire :  2.0 --> Option Value 6.815
Initial price : 42.0, Sigma : 0.20, Expire :  1.0 --> Option Value 1.589
Initial price : 42.0, Sigma : 0.20, Expire :  2.0 -